<a href="https://colab.research.google.com/github/tiffanytay/tiffanytay.github.io/blob/main/Powered_by_TNT_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install python libraries if not already installed
!pip install chatterbot
!pip install chatterbot_corpus

In [67]:
# Print file paths of generic ENGLISH corpus files
import os
import chatterbot_corpus

corpus_path = chatterbot_corpus.__path__[0]

for root, _, files in os.walk(corpus_path):
    for file in files:
      file_path = os.path.join(root, file)
      if "english" in file_path:
          print(file_path)

/usr/local/lib/python3.11/dist-packages/chatterbot_corpus/data/english/gossip.yml
/usr/local/lib/python3.11/dist-packages/chatterbot_corpus/data/english/movies.yml
/usr/local/lib/python3.11/dist-packages/chatterbot_corpus/data/english/money.yml
/usr/local/lib/python3.11/dist-packages/chatterbot_corpus/data/english/science.yml
/usr/local/lib/python3.11/dist-packages/chatterbot_corpus/data/english/psychology.yml
/usr/local/lib/python3.11/dist-packages/chatterbot_corpus/data/english/humor.yml
/usr/local/lib/python3.11/dist-packages/chatterbot_corpus/data/english/conversations.yml
/usr/local/lib/python3.11/dist-packages/chatterbot_corpus/data/english/politics.yml
/usr/local/lib/python3.11/dist-packages/chatterbot_corpus/data/english/food.yml
/usr/local/lib/python3.11/dist-packages/chatterbot_corpus/data/english/emotion.yml
/usr/local/lib/python3.11/dist-packages/chatterbot_corpus/data/english/trivia.yml
/usr/local/lib/python3.11/dist-packages/chatterbot_corpus/data/english/health.yml
/usr/

In [90]:
# Import packages
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
from chatterbot.trainers import ListTrainer

# Create and name chatbot
chatbot = ChatBot(
    'Powered by TNT Chatbot',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    database_uri='sqlite:///database.sqlite3',
    logic_adapters=[
        {
            'import_path': 'chatterbot.logic.BestMatch',
            'statement_comparison_function': 'chatterbot.comparisons.LevenshteinDistance',
            'response_selection_method': 'chatterbot.response_selection.get_first_response',
            'default_response': 'I am sorry, but I do not understand.',
            'maximum_similarity_threshold': 0.90
        },
        'chatterbot.logic.MathematicalEvaluation',
        'chatterbot.logic.TimeLogicAdapter'
    ]
)

In [91]:
# Test generic corpuses - 1
response = chatbot.get_response('Hi')
print(response)

I am sorry, but I do not understand.


In [92]:
# Test generic corpuses - 2
response = chatbot.get_response('Are you funny?')
print(response)

I am sorry, but I do not understand.


In [ ]:
# # Get custom corpus json from Github
# import urllib.request
# import json
# import pandas as pd

# # Raw url for custom corpus json file in Github
# url = 'https://raw.githubusercontent.com/tiffanytay/tiffanytay.github.io/refs/heads/main/projects/powered_by_tnt/custom_corpus_training_data.json'
# response = urllib.request.urlopen(url)
# # Save json data to 'data'
# data = json.loads(response.read())
# # Check 'data'
# print(data)

In [59]:
# # Import json trainer package
# from chatterbot.trainers import JsonFileTrainer

# # Create a new json trainer
# trainer_json = JsonFileTrainer(
#     chatbot,
#     field_map={
#        'persona': 'persona',
#        'text': 'text',
#        'conversation': 'conversation',
#        'in_response_to': 'in_response_to',
#     }
# )

# Put json data in a DataFrame
#df = pd.json_normalize(data, record_path="conversations")
#df.head()

In [93]:
# Read and parse custom corpus json file from Google Drive
import json

# Load your JSON file
with open("/content/drive/MyDrive/Colab Notebooks/powered_by_tnt_chatbot_training_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

conversations = data["conversations"]

In [100]:
# Create a new LIST trainer for the chatbot
trainer_list = ListTrainer(chatbot)  # chatbot is your ChatterBot instance

# Create a new GENERIC CORPUS trainer for the chatbot
trainer_corpus = ChatterBotCorpusTrainer(chatbot)

# Train the chatbot based on the GENERIC CORPORA: English greetings and conversations
trainer_corpus.train(
    "/usr/local/lib/python3.11/dist-packages/chatterbot_corpus/data/english/conversations.yml",
    "/usr/local/lib/python3.11/dist-packages/chatterbot_corpus/data/english/greetings.yml",
    "/usr/local/lib/python3.11/dist-packages/chatterbot_corpus/data/english/ai.yml",
    "/usr/local/lib/python3.11/dist-packages/chatterbot_corpus/data/english/botprofile.yml"
)

# Prepare the data for the LIST Trainer.
# The ListTrainer expects a list of strings where each string is a turn in the conversation.
# Based on the probable structure of your JSON data,
# let's assume each dictionary in conversations_data represents a turn.
# We need to flatten this structure into a simple list of strings.
# We'll take the 'text' field as the statement for training.

# This list will hold the sequence of statements for training.
conversation_list_for_training = []

# Iterate through the list of dictionaries and extract the text
for item in conversations:
    # Add the 'text' from each dictionary to the training list
    if 'text' in item:
        conversation_list_for_training.append(item['text'])
    # If your data has a back-and-forth structure within each dictionary,
    # you might need to extract multiple strings per dictionary.
    # For example, if 'in_response_to' also needs to be part of the training sequence.
    # However, the ListTrainer usually takes a linear sequence of statements.
    # If 'in_response_to' is the previous turn, you might add it before the current 'text'.
    # But the most common use case is a simple list of statements that follow each other.


# Train the chatbot with the flattened list of statements
# Ensure the list is not empty before training
if conversation_list_for_training:
    trainer_list.train(conversation_list_for_training)
else:
  print("No conversation data found to train with.")

ChatterBot Corpus Trainer: 4it [00:01,  2.75it/s]
List Trainer: 26it [00:00, 327.71it/s]


In [101]:
# Test custom corpus - 1
response = chatbot.get_response('Who are you?')
print(response)

Who? Who is but a form following the function of what


In [102]:
# Test custom corpus - 2
response = chatbot.get_response('Why should I hire you?')
print(response)

A lot of things, like all the other digits other than 0 and 1.


In [103]:
# Test custom corpus - 3
response = chatbot.get_response('Can you tell me about your experience in product management?')
print(response)

I ensure product-market fit by continuously gathering customer feedback and analyzing market trends. I prioritize features that align with user needs and business objectives to create a successful product.


In [104]:
# Test custom corpus - 4
response = chatbot.get_response('What do you like to do for fun?')
print(response)

What are you then?


In [105]:
# Test custom corpus - 5
response = chatbot.get_response('What time is it?')
print(response)

The current time is 08:49 PM


In [89]:
# Run chatbot continuously until stopped
while True:
    try:
        bot_input = chatbot.get_response(input())
        print(bot_input)

    except(KeyboardInterrupt, EOFError, SystemExit):
        break

Yo
Right this way.
Where are we going?
I am doing well.
1+2?
I am sorry, but I do not understand.
What time is it?
The current time is 08:43 PM
What do you do for fun?
I am doing well.
What are your hobbies?
Playing soccer, painting, and writing are my hobbies. How about you?
How do you approach product management?
I prioritize stakeholder management by maintaining open lines of communication and actively seeking input from all relevant parties. I believe that collaboration is key to successful product development.
